In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Dataset

In [2]:
!curl -O https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 80876  100 80876    0     0   292k      0 --:--:-- --:--:-- --:--:--  293k


In [3]:
orig_df = pd.read_csv('course_lead_scoring.csv')
df = orig_df.copy()

In [4]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In this dataset our desired target for classification task will be `converted` variable - has the client signed up to the platform or not.

## Data preparation
```
Check if the missing values are presented in the features.

If there are missing values:
- For categorical features, replace them with 'NA'
- For numerical features, replace with with 0.0
```

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1462 entries, 0 to 1461
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   lead_source               1334 non-null   object 
 1   industry                  1328 non-null   object 
 2   number_of_courses_viewed  1462 non-null   int64  
 3   annual_income             1281 non-null   float64
 4   employment_status         1362 non-null   object 
 5   location                  1399 non-null   object 
 6   interaction_count         1462 non-null   int64  
 7   lead_score                1462 non-null   float64
 8   converted                 1462 non-null   int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 102.9+ KB


In [6]:
df.columns

Index(['lead_source', 'industry', 'number_of_courses_viewed', 'annual_income',
       'employment_status', 'location', 'interaction_count', 'lead_score',
       'converted'],
      dtype='object')

Separating categorical and numerical columns

In [60]:
numerical = ['number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score']
categorical = ['lead_source', 'industry', 'employment_status', 'location']

Checking the missing values of Categorical and Numerical columns

In [8]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [9]:
df[categorical].isnull().sum()

lead_source          128
industry             134
employment_status    100
location              63
dtype: int64

In [10]:
df[numerical].isnull().sum()

number_of_courses_viewed      0
annual_income               181
interaction_count             0
lead_score                    0
dtype: int64

Filling the missing values of categorical columns with `NA` and Numerical with `0`

In [11]:
df[categorical] = df[categorical].fillna('NA')

In [12]:
df[numerical] = df[numerical].fillna(0.0)

In [13]:
df.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

### Question 1
```
What is the most frequent observation (mode) for the column industry?

- NA
- technology
- healthcare
- retail
```

In [14]:
df['industry'].value_counts()

industry
retail           203
finance          200
other            198
healthcare       187
education        187
technology       179
manufacturing    174
NA               134
Name: count, dtype: int64

In [15]:
df['industry'].mode()

0    retail
Name: industry, dtype: object

> The most frequent observation (mode) for the column industry is `retail`

### Question 2

- Create the correlation matrix for the numerical features of your dataset. In a correlation matrix, you compute the correlation coefficient between every pair of features.

- What are the two features that have the biggest correlation?
```
- interaction_count and lead_score
- number_of_courses_viewed and lead_score
- number_of_courses_viewed and interaction_count
- annual_income and interaction_count
- Only consider the pairs above when answering this question.
```

- Split the data
```
- Split your data in train/val/test sets with 60%/20%/20% distribution.
- Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.
- Make sure that the target value converted is not in your dataframe.
```

#### Correlation Matrix

In [16]:
numerical_df = df[numerical]
numerical_df.head()

,number_of_courses_viewed,annual_income,interaction_count,lead_score
0,1,79450.0,4,0.94
1,1,46992.0,1,0.80
2,5,78796.0,3,0.69
3,2,83843.0,1,0.87
4,3,85012.0,3,0.62


In [17]:
numerical_df.corr()

,number_of_courses_viewed,annual_income,interaction_count,lead_score
number_of_courses_viewed,1.000000,0.009770,-0.023565,-0.004879
annual_income,0.009770,1.000000,0.027036,0.015610
interaction_count,-0.023565,0.027036,1.000000,0.009888
lead_score,-0.004879,0.015610,0.009888,1.000000


#### Calculating the biggest correlation

In [18]:
corr_mat = numerical_df.corr().values
corr_mat

array([[ 1.        ,  0.00977029, -0.02356522, -0.004879  ],
       [ 0.00977029,  1.        ,  0.02703647,  0.01560955],
       [-0.02356522,  0.02703647,  1.        ,  0.00988818],
       [-0.004879  ,  0.01560955,  0.00988818,  1.        ]])

Removing diagonals

In [19]:
corr_mat = np.triu(corr_mat, k=1)
corr_mat

array([[ 0.        ,  0.00977029, -0.02356522, -0.004879  ],
       [ 0.        ,  0.        ,  0.02703647,  0.01560955],
       [ 0.        ,  0.        ,  0.        ,  0.00988818],
       [ 0.        ,  0.        ,  0.        ,  0.        ]])

In [20]:
idx_max = np.argmax(corr_mat)
idx_max

np.int64(6)

In [21]:
max_corr_val = corr_mat.flatten()[idx_max]
max_corr_val

np.float64(0.02703647240481443)

The two features that have the biggest correlation are `annual_income` and `interaction_count` with correlation co-efficient of `0.02703647240481443`

#### Split the data

In [22]:
from sklearn.model_selection import train_test_split

Splitting data in train/val/test sets with 60%/20%/20% distribution. First train/test --> 80/20

In [23]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

In [24]:
len(df_full_train), len(df_test)

(1169, 293)

```
dataset = 80% of full dataset
we want- 
- 60% of full data for train from this 80% data
- 20% of full data for validation from this 80% data

train: 
=> 80x/100 = 60
=> x = (60*100) / 80 = 75

val
=> 80x/100 = 20
=> x = (20*100) / 80 = 25

So, we have to split 0.75 and 0.25 for train and val from this 80% data
```
Splitting train/val --> 75/25; Overall, train/val/test --> 60/20/20

In [25]:
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [26]:
len(df_train), len(df_val), len(df_test)

(876, 293, 293)

#### Resetting index

In [27]:
df_train.reset_index(drop=True, inplace=True)
df_val.reset_index(drop=True, inplace=True)
df_test.reset_index(drop=True, inplace=True)

#### Creating target matrices and dropping target col from train/val/test

In [28]:
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

#### Dropping/Deleting the target column from train/val/test dataset

In [29]:
df_train = df_train.drop('converted', axis=1)
df_val = df_val.drop('converted', axis=1)
df_test = df_test.drop('converted', axis=1)

### Question 3
- Calculate the mutual information score between converted and other categorical variables in the dataset. Use the training set only.

- Round the scores to 2 decimals using round(score, 2).

- Which of these variables has the biggest mutual information score?
```
- industry
- location
- lead_source
- employment_status
```

#### Mutual information score between converted and other categorical variables in the Train set

In [30]:
# df_full_train

In [31]:
df_full_train[categorical]

,lead_source,industry,employment_status,location
1066,social_media,manufacturing,self_employed,australia
638,events,retail,student,north_america
799,social_media,education,NA,europe
380,referral,education,employed,australia
303,paid_ads,healthcare,employed,europe
...,...,...,...,...
1130,events,manufacturing,employed,south_america
1294,events,healthcare,student,south_america
860,paid_ads,manufacturing,student,north_america
1459,paid_ads,technology,student,north_america


In [32]:
from sklearn.metrics import mutual_info_score

In [33]:
mutual_info_score(df_full_train.industry, df_full_train.converted)

0.011684562750165564

Creating a function to find Mutual Info between all categorical column and target column

In [34]:
def mutual_info_converted_score(series):
    return mutual_info_score(df_full_train.converted, series)

In [35]:
df_full_train[categorical].apply(mutual_info_converted_score).sort_values(ascending=False)

lead_source          0.025665
employment_status    0.013258
industry             0.011685
location             0.002253
dtype: float64

`lead_source` has the highest Mutual Info score from all the categorical columns

### Question 4
Now let's train a logistic regression.

- Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.

- Fit the model on the training dataset.

- To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
```python
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
```
- Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

- What accuracy did you get?
```
a. 0.64
b. 0.74
c. 0.84
d. 0.94
```

Creating dictionary from our training dataset

In [36]:
train_dicts = df_train[categorical+numerical].to_dict(orient='records')

In [37]:
train_dicts[0]

{'lead_source': 'paid_ads',
 'industry': 'retail',
 'employment_status': 'student',
 'location': 'middle_east',
 'number_of_courses_viewed': 0,
 'annual_income': 58472.0,
 'interaction_count': 5,
 'lead_score': 0.03}

Initiating DictVectorizer for One-Hot encoding 

In [38]:
from sklearn.feature_extraction import DictVectorizer

In [39]:
dv = DictVectorizer(sparse=False)
dv

,dtype,<class 'numpy.float64'>
,separator,'='
,sparse,False
,sort,True


Creating X_train by fit_transform of DictVectorizer

In [40]:
X_train = dv.fit_transform(train_dicts)

In [41]:
X_train.shape

(876, 31)

Initiating and fitting the model

In [42]:
from sklearn.linear_model import LogisticRegression

In [43]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [44]:
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


Testing with validation dataset

In [45]:
val_dicts = df_val[categorical+numerical].to_dict(orient='records')

In [46]:
val_dicts[0]

{'lead_source': 'paid_ads',
 'industry': 'healthcare',
 'employment_status': 'unemployed',
 'location': 'europe',
 'number_of_courses_viewed': 3,
 'annual_income': 52220.0,
 'interaction_count': 1,
 'lead_score': 0.07}

In [47]:
X_val = dv.transform(val_dicts)

In [48]:
X_val.shape

(293, 31)

In [49]:
y_pred_val = model.predict_proba(X_val)

In [50]:
y_pred_val = y_pred_val[:, 1]
y_pred_val

array([0.61192163, 0.79982617, 0.53021344, 0.47131479, 0.57066131,
       0.44227168, 0.87127669, 0.84883115, 0.83290037, 0.61497801,
       0.54968026, 0.78153088, 0.69039785, 0.77017122, 0.5265944 ,
       0.91706425, 0.53170634, 0.42123048, 0.30146455, 0.84881583,
       0.79488653, 0.73670374, 0.44527211, 0.64838383, 0.4176882 ,
       0.75393418, 0.90166116, 0.33903048, 0.43181431, 0.9680681 ,
       0.92018714, 0.37487988, 0.652301  , 0.90650057, 0.75164116,
       0.64202121, 0.82250075, 0.83375553, 0.659116  , 0.30978853,
       0.78942264, 0.35546366, 0.96517758, 0.63389304, 0.51274195,
       0.53230533, 0.82287785, 0.744074  , 0.73452313, 0.68955217,
       0.46964443, 0.84539252, 0.55635243, 0.92637871, 0.65258021,
       0.61526273, 0.63816995, 0.28304018, 0.48049824, 0.57890618,
       0.35497342, 0.62175051, 0.38960778, 0.61156056, 0.85304278,
       0.75430136, 0.89185954, 0.71946459, 0.95387623, 0.89209517,
       0.75277087, 0.33850139, 0.61376593, 0.51622275, 0.64088

In [51]:
convertion_pred = (y_pred_val >= 0.5)

In [77]:
round((convertion_pred == y_val).mean(), 2)

np.float64(0.7)

In [78]:
original_acc = (convertion_pred == y_val).mean()
original_acc

np.float64(0.6996587030716723)

The accuracy from the validation data is `0.70`

In [56]:
# train_dicts_cat = df_train[categorical].to_dict(orient='records')
# dv_cat = DictVectorizer(sparse=False)
# X_train_cat = dv_cat.fit_transform(train_dicts_cat)
# model_cat = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
# model_cat.fit(X_train_cat, y_train)
# val_dicts_cat = df_val[categorical].to_dict(orient='records')
# X_val_cat = dv_cat.transform(val_dicts_cat)
# y_pred_val_cat = model_cat.predict_proba(X_val_cat)[:, 1]
# convertion_pred_cat = (y_pred_val_cat >= 0.5)
# round((convertion_pred_cat == y_val).mean(), 2)

### Question 5
Let's find the least useful feature using the feature elimination technique.

- Train a model using the same features and parameters as in Q4 (without rounding).

- Now exclude each feature from this set and train a model without it. Record the accuracy for each model.

- For each feature, calculate the difference between the original accuracy and the accuracy without the feature.

- Which of following feature has the smallest difference?
```
- 'industry'
- 'employment_status'
- 'lead_score'
```

In [57]:
features = ['industry', 'employment_status', 'lead_score']

Without `industry`

In [72]:
woi = ['employment_status', 'lead_score']

In [73]:
numerical

['number_of_courses_viewed',
 'annual_income',
 'interaction_count',
 'lead_score']

In [76]:
# df_train[woi+numerical]

In [79]:
train_dicts_cat = df_train[woi+numerical].to_dict(orient='records')

dv_cat = DictVectorizer(sparse=False)

X_train_cat = dv_cat.fit_transform(train_dicts_cat)

model_cat = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)

model_cat.fit(X_train_cat, y_train)

val_dicts_cat = df_val[woi+numerical].to_dict(orient='records')

X_val_cat = dv_cat.transform(val_dicts_cat)

y_pred_val_cat = model_cat.predict_proba(X_val_cat)[:, 1]

convertion_pred_cat = (y_pred_val_cat >= 0.5)

acc_woi = (convertion_pred_cat == y_val).mean()
acc_woi

C:\Users\User\AppData\Local\Temp\ipykernel_6824\3305540593.py:1: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  train_dicts_cat = df_train[woi+numerical].to_dict(orient='records')
C:\Users\User\AppData\Local\Temp\ipykernel_6824\3305540593.py:11: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  val_dicts_cat = df_val[woi+numerical].to_dict(orient='records')


np.float64(0.7030716723549488)

In [80]:
diff_industry = original_acc - acc_woi
diff_industry

np.float64(-0.0034129692832765013)

In [81]:
woe = ['industry', 'lead_score']
train_dicts_cat = df_train[woe+numerical].to_dict(orient='records')

dv_cat = DictVectorizer(sparse=False)

X_train_cat = dv_cat.fit_transform(train_dicts_cat)

model_cat = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)

model_cat.fit(X_train_cat, y_train)

val_dicts_cat = df_val[woe+numerical].to_dict(orient='records')

X_val_cat = dv_cat.transform(val_dicts_cat)

y_pred_val_cat = model_cat.predict_proba(X_val_cat)[:, 1]

convertion_pred_cat = (y_pred_val_cat >= 0.5)

acc_woe = (convertion_pred_cat == y_val).mean()
acc_woe

C:\Users\User\AppData\Local\Temp\ipykernel_6824\2823059701.py:2: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  train_dicts_cat = df_train[woe+numerical].to_dict(orient='records')
C:\Users\User\AppData\Local\Temp\ipykernel_6824\2823059701.py:12: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  val_dicts_cat = df_val[woe+numerical].to_dict(orient='records')


np.float64(0.689419795221843)

In [82]:
diff_es = original_acc - acc_woe
diff_es

np.float64(0.010238907849829282)

In [83]:
wol = ['industry', 'employment_status']
train_dicts_cat = df_train[wol+numerical].to_dict(orient='records')

dv_cat = DictVectorizer(sparse=False)

X_train_cat = dv_cat.fit_transform(train_dicts_cat)

model_cat = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)

model_cat.fit(X_train_cat, y_train)

val_dicts_cat = df_val[wol+numerical].to_dict(orient='records')

X_val_cat = dv_cat.transform(val_dicts_cat)

y_pred_val_cat = model_cat.predict_proba(X_val_cat)[:, 1]

convertion_pred_cat = (y_pred_val_cat >= 0.5)

acc_wol = (convertion_pred_cat == y_val).mean()
acc_wol

np.float64(0.7030716723549488)

In [84]:
diff_lead = original_acc - acc_wol
diff_lead

np.float64(-0.0034129692832765013)

In [85]:
print(diff_industry)
print(diff_es)
print(diff_lead)

-0.0034129692832765013
0.010238907849829282
-0.0034129692832765013


here we can see the least useful feature is: `employment_status` without it the accuracy increases, with it accuracy decreases to `0.6894`

### Question 6
Now let's train a regularized logistic regression.

- Let's try the following values of the parameter C: [0.01, 0.1, 1, 10, 100].
- Train models using all the features as in Q4.
- Calculate the accuracy on the validation dataset and round it to 3 decimal digits.
- Which of these C leads to the best accuracy on the validation set?
```
- 0.01
- 0.1
- 1
- 10
- 100
```

In [87]:
categorical + numerical

['lead_source',
 'industry',
 'employment_status',
 'location',
 'number_of_courses_viewed',
 'annual_income',
 'interaction_count',
 'lead_score']

In [88]:
C = [0.01, 0.1, 1, 10, 100]
accuracy_list = []
for c in C:
    train_dicts_cat = df_train[categorical+numerical].to_dict(orient='records')
    
    dv_cat = DictVectorizer(sparse=False)
    
    X_train_cat = dv_cat.fit_transform(train_dicts_cat)
    
    model_cat = LogisticRegression(solver='liblinear', C=c, max_iter=1000, random_state=42)
    
    model_cat.fit(X_train_cat, y_train)
    
    val_dicts_cat = df_val[categorical+numerical].to_dict(orient='records')
    
    X_val_cat = dv_cat.transform(val_dicts_cat)
    
    y_pred_val_cat = model_cat.predict_proba(X_val_cat)[:, 1]
    
    convertion_pred_cat = (y_pred_val_cat >= 0.5)
    
    acc = (convertion_pred_cat == y_val).mean()
    accuracy_list.append(acc)
    print(f"For C=({c}), Accuracy=({acc})")

For C=(0.01), Accuracy=(0.6996587030716723)
For C=(0.1), Accuracy=(0.6996587030716723)
For C=(1), Accuracy=(0.6996587030716723)
For C=(10), Accuracy=(0.6996587030716723)
For C=(100), Accuracy=(0.6996587030716723)


All given value of `C` has the same result `0.6996587030716723`. 